In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [3]:
alternative_words = {
    'fuck': 'function',
    'awful': 'array',
    'bitch': 'bert',
    'sexist': 'linkedlist',
    'shit': 'git',
    'slut': 'struct',
    'sexism': 'lexism',
    'whore': 'core',
    'kitchen': 'knn',
    'sex': 'sets',
    'hoe': 'hacker',
    'devil': 'dictionary',
    'hate' : 'hash'
}

In [4]:
import tensorflow as tf
import tensorflow.keras.layers as layers

model = Sequential()
model.add(Embedding(input_dim=30000, output_dim=20, input_length=20))
model.add(Bidirectional(LSTM(units=100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
dataset = pd.read_csv('sexismdataset.csv')
dataset.dropna(inplace=True)

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import re
def clean(text):

  cleanr = re.compile('<[^>]*>')           # remove html
  cleantext = re.sub(cleanr, ' ', text)

  cleantext = re.sub("[-]", " " , cleantext)   # remove - sign

  cleantext = re.sub("[^A-Za-z ]", " " , cleantext)  # remove evey character except alphabet
  cleantext = cleantext.lower()

  words = nltk.tokenize.word_tokenize(cleantext)
  words_new = [i for i in words if i not in stop_words]

  w = [lemmatizer.lemmatize(word) for word in words_new if len(word)>2]

  return ' '.join(w)

In [8]:
dataset['Text']=dataset['Text'].apply(clean)

In [12]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(dataset['Text'])
sequences = tokenizer.texts_to_sequences(dataset['Text'])
padded_sequences = pad_sequences(sequences, maxlen=20)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, dataset['oh_label'], epochs=10, batch_size=32)

Epoch 1/10
465/465 [==============================] - 16s 24ms/step - loss: 0.0327 - accuracy: 0.9888
Epoch 2/10
465/465 [==============================] - 11s 24ms/step - loss: 0.0247 - accuracy: 0.9919
Epoch 3/10
465/465 [==============================] - 11s 24ms/step - loss: 0.0202 - accuracy: 0.9932
Epoch 4/10
465/465 [==============================] - 12s 26ms/step - loss: 0.0177 - accuracy: 0.9939
Epoch 5/10
465/465 [==============================] - 13s 28ms/step - loss: 0.0158 - accuracy: 0.9952
Epoch 6/10
465/465 [==============================] - 13s 28ms/step - loss: 0.0143 - accuracy: 0.9957
Epoch 7/10
465/465 [==============================] - 13s 27ms/step - loss: 0.0150 - accuracy: 0.9950
Epoch 8/10
465/465 [==============================] - 13s 28ms/step - loss: 0.0133 - accuracy: 0.9956
Epoch 9/10
465/465 [==============================] - 13s 27ms/step - loss: 0.0117 - accuracy: 0.9960
Epoch 10/10
465/465 [==============================] - 13s 27ms/step - loss: 0.010

In [13]:
def replace_curse_words(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=20)
    
    prediction = model.predict(padded_sequence)[0][0]
    if prediction < 0.5:
        return text
    
    for curse_word, alternative_word in alternative_words.items():
        text = text.replace(curse_word, alternative_word)
    
    return text

sample=input('be a sexist or not, space is yours ')
replace_curse_words(sample)

be a sexist or not, space is yours this bitch is a slut
1/1 [==============================] - 0s 374ms/step


'this bert is a struct'